In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
synthetic_df = pd.read_csv('/content/drive/Shareddrives/CMPUT 664 : SSE/Data/Pima Diabetes/synthetic_pima_diabetic.csv')

In [3]:
x = synthetic_df.drop('Outcome',axis = 1 )
y = synthetic_df['Outcome'].to_numpy()
x = pd.get_dummies(x).iloc[:].to_numpy()

In [4]:
from typing import List, Tuple, Dict
from copy import copy

from tqdm import tqdm_notebook

from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.base import clone, BaseEstimator

try:
    import tensorflow as tf
except (ModuleNotFoundError, ImportError):
    import warnings

    warnings.warn("Tensorflow is not installed")

class ShadowModels:
    """
    Creates a swarm of shadow models and trains them with a split
    of the synthetic data.
    Parameters
    ----------
    X: ndarray or DataFrame
    y: ndarray or str
        if X it's a DataFrame then y must be the target column name,
        otherwise 
    n_models: int
        number of shadow models to build. Higher number returns
        better results but is limited by the number of records 
        in the input data.
    target_classes: int
        number of classes of the target model or lenght of the
        prediction array of the target model.
    learner: learner? #fix type
        learner to use as shadow model. It must be as similar as 
        possible to the target model. It must have `predict_proba` 
        method. Now only sklearn learners are implemented.
    Returns
    -------
    ShadowModels object
    """

    def __init__(
        self,
        X: np.ndarray,
        y: np.ndarray,
        n_models: int,
        target_classes: int,
        learner,
        **fit_kwargs,
    ) -> None:

        self.n_models = n_models
        self.X = X
        if self.X.ndim > 1:
            # flatten images or matrices inside 1rst axis
            self.X = self.X.reshape(self.X.shape[0], -1)

        self.y = y
        self.target_classes = target_classes
        self._splits = self._split_data(self.X, self.y, self.n_models, self.target_classes)
        self.learner = learner
        self.models = self._make_model_list(self.learner, self.n_models)

        # train models
        self.results = self.train_predict_shadows(**fit_kwargs)

    @staticmethod
    def _split_data(
        X: np.ndarray, y: np.ndarray, n_splits: int, n_classes: int
    ) -> List[np.ndarray]:
        """
        Split manually into n datasets maintaining class proportions
        """
        # data = np.hstack((data[0], data[1].reshape(-1, 1)))
        # X = data
        # y = data[:, -1]
        classes = range(n_classes)
        class_partitions = []
        # Split by class
        for clss in classes:

            X_clss = X[y == clss]
            y_clss = y[y == clss]
            batch_size = len(X_clss) // n_splits
            splits = []
            for i in range(n_splits):
                split_X = X_clss[i * batch_size : (i + 1) * batch_size, :]
                split_y = y_clss[i * batch_size : (i + 1) * batch_size]
                splits.append(np.hstack((split_X, split_y.reshape(-1, 1))))
            class_partitions.append(splits)

        # -------------------
        # consolidate splits into ndarrays
        # -------------------

        grouped = []
        for split in range(n_splits):
            parts = []
            for part in class_partitions:
                parts.append(part[split])
            grouped.append(parts)

        splits = []
        for group in grouped:
            splits.append(np.vstack(group))

        return splits

    @staticmethod
    def _make_model_list(learner, n) -> List:
        """
        Intances n shadow models, copies of the input parameter learner
        """
        try:
            if isinstance(learner, tf.keras.models.Model):
                models = [copy(learner) for _ in range(n)]
        except NameError:
            print("using sklearn shadow models")
            pass

        if isinstance(learner, BaseEstimator):
            models = [clone(learner) for _ in range(n)]

        return models

    def train_predict_shadows(self, **fit_kwargs):
        """
        "in" : 1
        "out" : 0
        """

        # TRAIN and predict
        results = []
        for model, data_subset in tqdm_notebook(zip(self.models, self._splits)):
            X = data_subset[:, :-1]
            y = data_subset[:, -1]
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

            model.fit(X_train, y_train, **fit_kwargs)
            # data IN training set labelet 1
            y_train = y_train.reshape(-1, 1)
            predict_in = model.predict_proba(X_train)
            res_in = np.hstack((predict_in, y_train, np.ones_like(y_train)))

            # data OUT of training set, labeled 0
            y_test = y_test.reshape(-1, 1)
            predict_out = model.predict_proba(X_test)
            print(predict_out)
            res_out = np.hstack((predict_out, y_test, np.zeros_like(y_test)))

            # concat in single array
            model_results = np.vstack((res_in, res_out))
            results.append(model_results)

        results = np.vstack(results)
        return results

    def __repr__(self):
        rep = (
            f"Shadow models: {self.n_models}, {self.learner.__class__}\n"
            f"lengths of data splits : {[len(s) for s in self._splits]}"
        )
        return rep

In [5]:
logisticRegr = LogisticRegression(solver='liblinear')
sh = ShadowModels(x, y, 5, 3, logisticRegr)
shadow_data = sh.results

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:134: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


0it [00:00, ?it/s]

[[0.57751633 0.42248367]
 [0.63996033 0.36003967]
 [0.83751457 0.16248543]
 [0.23321613 0.76678387]
 [0.69836577 0.30163423]
 [0.91251825 0.08748175]
 [0.72636764 0.27363236]
 [0.80627859 0.19372141]
 [0.16473907 0.83526093]
 [0.62577784 0.37422216]
 [0.8572738  0.1427262 ]
 [0.90461818 0.09538182]
 [0.76444607 0.23555393]
 [0.46209214 0.53790786]
 [0.80880549 0.19119451]
 [0.52212523 0.47787477]
 [0.19900114 0.80099886]
 [0.21350305 0.78649695]
 [0.69876493 0.30123507]
 [0.77490282 0.22509718]
 [0.77645427 0.22354573]
 [0.3804479  0.6195521 ]
 [0.83982078 0.16017922]
 [0.97968249 0.02031751]
 [0.53802899 0.46197101]
 [0.88883535 0.11116465]
 [0.34070654 0.65929346]
 [0.60271475 0.39728525]
 [0.70208009 0.29791991]
 [0.56855453 0.43144547]
 [0.61506348 0.38493652]
 [0.85295034 0.14704966]
 [0.60861864 0.39138136]
 [0.69632456 0.30367544]
 [0.59177965 0.40822035]
 [0.94623685 0.05376315]
 [0.49194723 0.50805277]
 [0.509154   0.490846  ]
 [0.78112023 0.21887977]
 [0.68455684 0.31544316]


In [6]:
shadow_data

array([[0.39433164, 0.60566836, 0.        , 1.        ],
       [0.90077427, 0.09922573, 0.        , 1.        ],
       [0.84538132, 0.15461868, 0.        , 1.        ],
       ...,
       [0.76204273, 0.23795727, 1.        , 0.        ],
       [0.34760641, 0.65239359, 0.        , 0.        ],
       [0.71647739, 0.28352261, 0.        , 0.        ]])

In [7]:
class AttackModels:
    def __init__(self, target_classes, attack_learner):
        """
        Attacker models to learn class membership from shadow data.
        Parameters
        ----------
        target_classes: int
            number of classes that the target model can predict
        attack_learning: learner
            trainable learner to model memebership from shadow data.
            The learner its cloned into n models, one for each target class,
            and each model is trained on a class subset of the shadow data.
        Returns
        -------
        AttackModels class instance
        """
        self.target_classes = target_classes
        self.attack_learner = attack_learner
        # 1 model for each class
        self.attack_models = [clone(self.attack_learner) for _ in range(target_classes)]

        self._fited = False

    @staticmethod
    def _update_learner_params(learner, **learner_params) -> None:
        # safety check if dict is well formed
        for k in learner_params.keys():
            if not hasattr(learner, k):
                raise AttributeError(
                    f"Learner parameter {k} is not an attribute of {learner.__class__}"
                )

        # update learner params
        learner.__dict__.update(**learner_params)

    def fit(self, shadow_data, **learner_kwargs) -> None:
        """
        Trains `attack_models` with `shadow_data`. Each model is trained with
        with a subset of the same class of `shadow_data`.
        Parameters
        ----------
        shadow_data: np.ndarray
            Shadow data. Results from `ShadowModels`.
            Last column (`[:,-1]`) must be the membership label of the shadow
            prediction, where 1 means that the record was present in the 
            shadow training set ('in') and 0 if the recored was in the test
            set ('out').
            Second last column (`[:,-2]`) must be the data class. this will
            be used as grouper to split the data for each attack model.
            The rest of the columns are the class probability vector
            predicted by the shadow model.
        Returns
        -------
        None
        TODO
        ----
            Tweak model params with something like **learner_kwargs
            cross-validate
            grid search?
        """
        # split data into subsets, n == target_classes
        membership_label = shadow_data[:, -1]
        class_label = shadow_data[:, -2]
        data = shadow_data[:, :-2]
        for i, model in enumerate(self.attack_models):
            X = data[class_label == i]
            y = membership_label[class_label == i]

            # update model params
            self._update_learner_params(model, **learner_kwargs)
            # train model
            model.fit(X, y)

        self._fited = True

    def predict(self, X, y, batch=False) -> np.ndarray:
        """
        Predicts if `X` is real member of `y` in the attacked
        private training set.
        Parameters
        ----------
        X: np.ndarray
            Probability vector result from target model
        y: int, np.ndarray
            estimated class of the data record used to get `X`
        """
        if not self._fited:
            print("Must run `fit` method first")
            return

        if not batch:
            model_cls = y
            model = self.attack_models[model_cls]
            prob_vec = model.predict_proba(X)

            if y == np.argmax(prob_vec) and np.argmax(prob_vec) == 1:
                return 1

            else:
                return 0

        elif batch:

            model_classes = np.unique(y)
            res = []
            for model_cls in model_classes:
                X_cls = X[y == model_cls]
                model = self.attack_models[model_cls]
                attack_res = model.predict_proba(X_cls)
                res.append(attack_res)

            return np.concatenate(res)

In [8]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100)
attacker = AttackModels(target_classes=2, attack_learner=model)
attacker.fit(shadow_data)

## Membership attack on decision tree

In [51]:
df =  pd.read_csv('/content/drive/Shareddrives/CMPUT 664 : SSE/Data/Pima Diabetes/diabetes.csv')
x = df.drop('Outcome',axis = 1 )
y = df['Outcome']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.34, random_state=42)
clf = DecisionTreeClassifier(random_state=0,max_leaf_nodes = 50).fit(x_train,y_train)

In [52]:
y_pred_train = clf.predict(x_train)
train_a = accuracy_score(y_train, y_pred_train, normalize=True)*100
print('accuracy on train data is',train_a)

y_pred_test = clf.predict(x_test)
test_a = accuracy_score(y_test, y_pred_test, normalize=True)*100
print('accuracy on test data is',test_a)

accuracy on train data is 95.0592885375494
accuracy on test data is 70.99236641221374


In [53]:
X_in = clf.predict_proba(x_train)
res_in = attacker.predict(X_in, y_train, batch=True)

In [54]:
X_in

array([[0.9047619 , 0.0952381 ],
       [0.98113208, 0.01886792],
       [0.96969697, 0.03030303],
       ...,
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.92307692, 0.07692308]])

In [55]:
res_in

array([[0.8 , 0.2 ],
       [0.12, 0.88],
       [0.76, 0.24],
       ...,
       [0.97, 0.03],
       [0.97, 0.03],
       [0.97, 0.03]])

In [56]:
X_out = clf.predict_proba(x_test)
res_out = attacker.predict(X_out, y_test, batch=True)

In [57]:
from sklearn.metrics import precision_score, recall_score, f1_score
y_pred = np.concatenate((np.argmax(res_in, axis=1), np.argmax(res_out, axis=1)))
y_true = np.concatenate((np.ones_like(y_train), np.zeros_like(y_test)))

In [58]:
precision_score(y_true, y_pred)

0.6780626780626781

In [59]:
recall_score(y_true, y_pred)

0.47035573122529645

In [60]:
f1_score(y_true, y_pred)

0.5554259043173861

# Membership attack on target Random Classifier

In [87]:
from sklearn.ensemble import RandomForestClassifier
r_clf = RandomForestClassifier(n_estimators = 10,random_state=0)
r_clf.fit(x_train, y_train)

RandomForestClassifier(n_estimators=10, random_state=0)

In [88]:
y_pred_train = r_clf.predict(x_train)
train_a = accuracy_score(y_train, y_pred_train, normalize=True)*100
print('accuracy on train data is',train_a)

y_pred_test = r_clf.predict(x_test)
test_a = accuracy_score(y_test, y_pred_test, normalize=True)*100
print('accuracy on test data is',test_a)

accuracy on train data is 98.41897233201581
accuracy on test data is 75.19083969465649


In [89]:
X_in = r_clf.predict_proba(x_train)
res_input = attacker.predict(X_in, y_train, batch=True)

In [90]:
X_out = r_clf.predict_proba(x_test)
res_output = attacker.predict(X_out, y_test, batch=True)

In [91]:
y_pred = np.concatenate((np.argmax(res_input, axis=1), np.argmax(res_output, axis=1)))
y_true = np.concatenate((np.ones_like(y_train), np.zeros_like(y_test)))

In [92]:
precision_score(y_true, y_pred)

0.597444089456869

In [93]:
recall_score(y_true, y_pred)

0.3695652173913043

In [26]:
f1_score(y_true, y_pred)

0.22945736434108524

# Membership inference attack on stacking classifier

In [67]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import StackingClassifier

estimators = [
    ('rf', RandomForestClassifier(n_estimators=5, random_state=42)),
    ('svr', make_pipeline(StandardScaler(),
                          LinearSVC(random_state=42)))
]
s_clf = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression()
)

s_clf.fit(x_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number

StackingClassifier(estimators=[('rf',
                                RandomForestClassifier(n_estimators=5,
                                                       random_state=42)),
                               ('svr',
                                Pipeline(steps=[('standardscaler',
                                                 StandardScaler()),
                                                ('linearsvc',
                                                 LinearSVC(random_state=42))]))],
                   final_estimator=LogisticRegression())

In [68]:
y_pred_train = s_clf.predict(x_train)
train_a = accuracy_score(y_train, y_pred_train, normalize=True)*100
print('accuracy on train data is',train_a)

y_pred_test = s_clf.predict(x_test)
test_a = accuracy_score(y_test, y_pred_test, normalize=True)*100
print('accuracy on test data is',test_a)

accuracy on train data is 81.22529644268775
accuracy on test data is 73.2824427480916


In [69]:
X_in = s_clf.predict_proba(x_train)
r_input = attacker.predict(X_in, y_train, batch=True)

In [70]:
X_out = s_clf.predict_proba(x_test)
r_output = attacker.predict(X_out, y_test, batch=True)

In [71]:
y_pred = np.concatenate((np.argmax(r_input, axis=1), np.argmax(r_output, axis=1)))
y_true = np.concatenate((np.ones_like(y_train), np.zeros_like(y_test)))

In [72]:
precision_score(y_true, y_pred)

0.6469135802469136

In [73]:
recall_score(y_true, y_pred)

0.5177865612648221

In [74]:
f1_score(y_true, y_pred)

0.575192096597146